# Mapping

Mappings are pre-defined configuration files that encode the logic on how to transform a specific data source into Resources that follow a template of a targeted _Type_. 

This notebook demonstrates the `DictionaryMapping` wich is based on a JSON structure that represent the target structure, and Python code that will apply desired transformations on the data source.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](./00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
#Let get some SHACL shapes from https://github.com/INCF/neuroshapes.git
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/activity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/entity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/ontology" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/person" "./neuroshapes/shapes/neurosciencegraph/commons/" 

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

Note: commented lines are not implemented on the RdfModel

## Imports

In [ ]:
from kgforge.core import Resource

In [ ]:
from kgforge.specializations.mappings import DictionaryMapping

## Data

In [ ]:
scientists = [
    {
        "id": 123,
        "name": "Marie Curie",
        "gender": "female",
        "middle_name": "Salomea",
    },
    {
        "id": 456,
        "name": "Albert Einstein",
        "gender": "male",
        "middle_name": "(missing)",
    },
]

## Mapping data to the Knowledge Graph Schema

### basics

In [ ]:
forge.types()

In [ ]:
forge.template("Dataset", only_required=True)

In [ ]:
mapping_simple = DictionaryMapping("""
    type: Dataset
    contribution:
    {
        type: Person
        id: forge.format("identifier", "persons", x.id)
        name: x.name
    }
""")

In [ ]:
mapping_simple = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
    }
""")

In [ ]:
resources_simple = forge.map(scientists, mapping_simple)

In [ ]:
print(resources_simple[0])

### missing values

In [ ]:
mapping_na = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
        additionalName: x.middle_name
    }
""")

In [ ]:
print(forge.map(scientists[1], mapping_na))

In [ ]:
print(forge.map(scientists[1], mapping_na, na="(missing)"))

### multiple mappings

In [ ]:
mapping_person = DictionaryMapping("""
    id: forge.format("identifier", "persons", x.id)
    type: Person
    name: x.name
""")

In [ ]:
mapping_association = DictionaryMapping("""
    type: Association
    agent: forge.format("identifier", "persons", x.id)
""")

In [ ]:
resources_graph = forge.map(scientists, [mapping_person, mapping_association])

In [ ]:
print(resources_graph[1])

In [ ]:
print(resources_graph[1])

### managed mappings

In [ ]:
forge.sources()

In [ ]:
forge.mappings("scientists-database")

In [ ]:
# mapping = forge.mapping("Association", "scientists-database")

In [ ]:
# resources = forge.map(scientists, mapping, na="(missing)")

In [ ]:
# type(resources)

In [ ]:
# type(resources[0])

In [ ]:
# print(mapping)

In [ ]:
# print(resources[0])

In [ ]:
# forge.register(resources)

## Managing mappings

In [ ]:
filepath = "mappings/scientists-database/DictionaryMapping/Association.hjson"

### saving

In [ ]:
# mapping.save(filepath)

### tracking & sharing changes

In [ ]:
# ! cd mappings

In [ ]:
# ! git add Association.hjson

In [ ]:
# ! git commit -m "Add Association mapping"

In [ ]:
# ! git push

### loading

In [ ]:
loaded = DictionaryMapping.load(filepath)

In [ ]:
# loaded == mapping